<a href="https://colab.research.google.com/github/VictorCanLima/BasicOpenGL/blob/master/FinalProjectAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Neural Network**
## **Final Project**
### Computer Systems Engineering
### LIS3082-1 Artificial Intelligence
### By Victor Armando Canales Lima (162328)
### Professor Gerardo Ayala San Martín
### Department of Computing, Electronics and Mechatronics
### Universidad de las Am ́ericas Puebla, San Andr ́es Cholula, Puebla, México
### May 14, 2021






In this code we show the application of a Simple Neural Network to classify eeg motor imagery signals (left hand movement and right hand movement)

First we do is to install the libraries we will need, make sure we can access the data tests. A folder with data samples should be given with this code. Put the folder in your Google Drive if you want to test this notebook from Google Colab, or leave it in the same directory than the notebook or python file you use to run this code.

## Data Preprocessing

In [ ]:
!pip install numpy matplotlib scipy numba scikit-learn mne PyWavelets pandas
!pip install mne-features

     |████████████████████████████████| 7.0MB 5.0MB/s 


In [ ]:
from mne_features.univariate import compute_hjorth_complexity_spect as hjorthComp
from mne_features.univariate import compute_hjorth_mobility_spect as hjorthMob
from mne_features.univariate import compute_ptp_amp as ptp_amp
from sklearn.model_selection import train_test_split as tts
from scipy.io import loadmat as load
from scipy import signal
import pandas as pd
import numpy as np


try:#If Running in a Google Colaboratory Notebook
  from google.colab import drive
  drive.mount('/content/drive')
  rawdatapath = '/content/drive/MyDrive/data/S4.mat'
except:#If Running in a Local Jupyter Notebook
  rawdatapath = 'data/S4.mat'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We create a class to import and preprocess the signal, we apply filters to clean the brain signal samples, we have 2 classes, right hand movement and left hand movement. 

In [ ]:
class EEG_Signal_Handler():
    def __init__(self, datapath):
        self.data = load(datapath)  # Importando datos
        self.C1=np.array(self.data['C1'])  # Movimiento Mano Izquierda 1
        self.C2=np.array(self.data['C2'])  # Movimiento Mano Derecha 2
        self.channels=len(self.C1[:,0,0])  # Numero de canales
        self.samples=len(self.C1[0,:,0])   # Número de muestras por experimento
        temp = np.arange(abs(len(self.C1[0,0,:])-len(self.C2[0,0,:])))
        if len(self.C1[0,0,:]) > len(self.C2[0,0,:]):
            self.C1 = np.delete(self.C1,temp,axis=2)
        else:
            self.C2 = np.delete(self.C2,temp,axis=2)
        self.experiments=len(self.C1[0,0,:])
        self.filter('highpass', 1)
        self.filter('lowpass', 30)
    def filter(self,filttype,cutfreq):
        Wn = cutfreq/(250/2)
        num, den = signal.butter(5,Wn,filttype)
        for i in range(self.experiments):
            self.C1[:,:,i] = signal.filtfilt(num,den,self.C1[:,:,i],1)
            self.C2[:,:,i] = signal.filtfilt(num,den,self.C2[:,:,i],1)

In [ ]:
mysig = EEG_Signal_Handler(rawdatapath)

## Feature Extraction

Now we extract the features we are going to use. We have seven different features, but as we have 3 channels (eeg sensors), we have 3 different signals, in this class, we create our datasets to train and test our model. We will have 21 different inputs for our model, one for each channel and one for each feature we extract from the signal. Some of the features are extracted from the library developed by  MNE-Features Developers.

Extracted features are:


1.   Root Mean Value
2.   Variance
3.   Standard Deviation
4.   Mean
5.   Hjorth Mobility
6.   Hjorth Complexity
7.   Peak to Peak Amplitude



In [ ]:
class Dataset_Creator():
    def __init__(self,class1,class2):
        featsC1 = self.get_features(class1)
        featsC2 = self.get_features(class2)
        labelsC1 = np.ones(mysig.experiments)
        labelsC2 = np.ones(mysig.experiments)
        self.x_complete = np.concatenate((featsC1,featsC2),axis=0)
        self.y_complete = np.concatenate((labelsC1,labelsC2))
        self.y_complete = np.concatenate((labelsC1,labelsC2))
        self.x_train, self.x_test = None, None
        self.y_train, self.y_test = None, None
        self.test_split()
    def test_split(self):
        X,x,Y,y = tts(self.x_complete,self.y_complete,test_size=0.2) 
        self.x_train, self.x_test = X,x
        self.y_train, self.y_test = Y,y
    def get_features(self,c):
        feats = np.empty((mysig.experiments,21))
        for i in range(mysig.experiments):
            feats[i,0] = np.sqrt(np.mean(c[0,:,i]**2))
            feats[i,1] = np.sqrt(np.mean(c[1,:,i]**2))
            feats[i,2] = np.sqrt(np.mean(c[2,:,i]**2))
            feats[i,3] = np.var(c[0,:,i])
            feats[i,4] = np.var(c[1,:,i])
            feats[i,5] = np.var(c[2,:,i])
            feats[i,6] = np.std(c[0,:,i])
            feats[i,7] = np.std(c[1,:,i])
            feats[i,8] = np.std(c[2,:,i])
            feats[i, 9] = np.mean(c[0,:,i])
            feats[i,10] = np.mean(c[1,:,i])
            feats[i,11] = np.mean(c[2,:,i])
            feats[i,12] = hjorthMob(250,c[0,:,i])
            feats[i,13] = hjorthMob(250,c[1,:,i])
            feats[i,14] = hjorthMob(250,c[2,:,i])
            feats[i,15] = hjorthComp(250,c[0,:,i])
            feats[i,16] = hjorthComp(250,c[1,:,i])
            feats[i,17] = hjorthComp(250,c[2,:,i])
            feats[i,18] = ptp_amp(c[0,:,i])
            feats[i,19] = ptp_amp(c[1,:,i])
            feats[i,20] = ptp_amp(c[2,:,i])
        return feats

In [ ]:
myfeatures = Dataset_Creator(mysig.C1,mysig.C2)

## Defining Neural Network

Now we define our Neural Network, inspired in the tutorial of the official YouTube channel of TensorFlow You. Our model will have 2 hiden layers, Rectifier activation function and as decision fuction for classification, the sigmoid fuction. As error function, we pick a binary crossentropy function, an as gain function, RMSprop, a gradient based optimization technique.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
classifier = Sequential() # Initialising the ANN

classifier.add(Dense(units = 11, activation = 'relu', input_dim = 21))
classifier.add(Dense(units = 6, activation = 'relu'))
classifier.add(Dense(units = 3, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
classifier.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy')

## Trainning

In [ ]:
classifier.fit(myfeatures.x_train, myfeatures.y_train, batch_size = 1, epochs = 100)

Epoch 1/100
316/316 [==============================] - 1s 1ms/step - loss: 9634.6702
Epoch 2/100
316/316 [==============================] - 0s 1ms/step - loss: 20.1148
Epoch 3/100
316/316 [==============================] - 0s 1ms/step - loss: 0.6192
Epoch 4/100
316/316 [==============================] - 0s 1ms/step - loss: 6.9944e-32
Epoch 5/100
316/316 [==============================] - 0s 1ms/step - loss: 1.2891e-33
Epoch 6/100
316/316 [==============================] - 0s 1ms/step - loss: 1.3675e-33
Epoch 7/100
316/316 [==============================] - 0s 1ms/step - loss: 2.6159e-33
Epoch 8/100
316/316 [==============================] - 0s 1ms/step - loss: 2.1839e-33
Epoch 9/100
316/316 [==============================] - 0s 1ms/step - loss: 2.9713e-32
Epoch 10/100
316/316 [==============================] - 0s 1ms/step - loss: 1.0692e-32
Epoch 11/100
316/316 [==============================] - 0s 1ms/step - loss: 2.1686e-32
Epoch 12/100
316/316 [==============================] - 0s 1

## Testing

In [ ]:
Y_pred = classifier.predict(myfeatures.x_test)
Y_pred = [ 1 if y>=0.5 else 0 for y in Y_pred ]

total = 0
correct = 0
wrong = 0
for i in Y_pred:
  total=total+1
  if(myfeatures.y_test[i] == Y_pred[i]):
    correct=correct+1
  else:
    wrong=wrong+1

print("Total " + str(total))
print("Correct " + str(correct))
print("Wrong " + str(wrong))

Total 80
Correct 80
Wrong 0


As we can see, we have a 100% accuracy. This is very hard to get with EEG signals, but the use of multiple features and the power of neural networks allow this to happen.

NeverthlesNevertheless, for real life application, this is not really a good option as Neural Networks implie high computational costs. 